## Preparing a dataset for centre back model
Aim is to collect a dataset on match lineups (defenders specifically) and match stats, specifically xG, goals and also touches in box / final 3rd.

### 1. Set up / dependencies

In [25]:
from statsbombpy import sb
import numpy as np
import pandas as pd

### 2. Read in data

In [27]:
# get the competition / season_ids
comps = sb.competitions()
comps = comps[comps.competition_id == 37]
comps.season_id

credentials were not supplied. open data access only


15    90
16    42
17     4
Name: season_id, dtype: int64

In [32]:
# get all the match ids
matches1 = sb.matches(competition_id=37, season_id=4)
matches2 = sb.matches(competition_id=37, season_id=42)
matches3 = sb.matches(competition_id=37, season_id=90)
matches = pd.concat([matches1, matches2, matches3])
match_ids = matches.match_id.values

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only


In [46]:
# get the lineup data
lineups = pd.DataFrame()
for m in match_ids:
    lineup = sb.lineups(match_id=m)
    teams = list(lineup.keys())
    team1 = teams[0]
    team2 = teams[1]
    lineup1 = pd.DataFrame(lineup[team1])
    lineup2 = pd.DataFrame(lineup[team2])
    lineup1["team"] = team1
    lineup2["team"] = team2  
    lineup1["match_id"] = m
    lineup2["match_id"] = m
    lineups = pd.concat([lineups, lineup1, lineup2])

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data acces

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data acces

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only


In [81]:
# extract key info to new df columns
lineups["position"] = lineups["positions"].apply(lambda x: x[0]["position"] if len(x) > 0 else None)
lineups["start_time"] = lineups["positions"].apply(lambda x: x[0]["from"] if len(x) > 0 else None)
lineups["end_time"] = lineups["positions"].apply(lambda x: x[0]["to"] if len(x) > 0 else None)

In [92]:
# only really interested in centre backs
print(lineups.position.unique())
cbs = ["Left Center Back", "Right Center Back", "Center Back"]
cb_lineups = lineups[lineups.position.isin(cbs)]

['Left Center Back' 'Left Center Forward' 'Left Center Midfield'
 'Goalkeeper' 'Right Center Forward' 'Right Center Back' 'Center Midfield'
 'Center Back' 'Right Center Midfield' 'Left Midfield' 'Center Forward'
 'Right Midfield' 'Left Wing' 'Right Back' 'Left Back' 'Right Wing'
 'Center Attacking Midfield' None 'Left Defensive Midfield'
 'Right Defensive Midfield' 'Center Defensive Midfield'
 'Left Attacking Midfield' 'Right Attacking Midfield' 'Secondary Striker'
 'Right Wing Back' 'Left Wing Back']


In [93]:
cb_lineups.shape

(1476, 12)

In [108]:
# get event data related to matches
events = sb.events(match_id=match_ids[0])

credentials were not supplied. open data access only


In [109]:
events.shape

(3793, 35)

In [110]:
events.sample(5)

,bad_behaviour,ball_receipt,ball_recovery,block,carry,counterpress,dribble,duel,duration,foul_committed,...,possession_team,related_events,second,shot,substitution,tactics,team,timestamp,type,under_pressure
1017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Manchester City WFC,"[30041b72-609a-4bfb-bb5a-2ee981b8fdbb, 30dca49...",57,NaN,NaN,NaN,Manchester City WFC,00:01:57.793,Ball Receipt*,True
2015,NaN,NaN,NaN,NaN,"{'end_location': [35.0, 41.0]}",NaN,NaN,NaN,1.213,NaN,...,Manchester City WFC,"[c4ca8edd-5be4-4b6e-bfd8-45a1d00656d8, dfe747e...",27,NaN,NaN,NaN,Manchester City WFC,00:09:27.700,Carry,NaN
3388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Chelsea FCW,NaN,34,NaN,NaN,NaN,Chelsea FCW,00:16:34.838,Ball Recovery,NaN
2316,NaN,NaN,NaN,NaN,"{'end_location': [68.0, 13.0]}",NaN,NaN,NaN,4.320,NaN,...,Chelsea FCW,"[09f33b45-e25a-4f4a-a04f-63a530cff412, 9e278a3...",27,NaN,NaN,NaN,Chelsea FCW,00:43:27.100,Carry,True
738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.040,NaN,...,Chelsea FCW,[eeb4f585-3ec0-4bee-8ef8-909ee1f8df41],47,NaN,NaN,NaN,Chelsea FCW,00:19:47.998,Pass,NaN


In [98]:
events.columns

Index(['bad_behaviour', 'ball_receipt', 'ball_recovery', 'block', 'carry',
       'counterpress', 'dribble', 'duel', 'duration', 'foul_committed',
       'foul_won', 'goalkeeper', 'id', 'index', 'injury_stoppage',
       'interception', 'location', 'match_id', 'minute', 'pass', 'period',
       'play_pattern', 'player', 'position', 'possession', 'possession_team',
       'related_events', 'second', 'shot', 'substitution', 'tactics', 'team',
       'timestamp', 'type', 'under_pressure'],
      dtype='object')

In [120]:
# get shot data
shots = events[events.shot.notnull()]
shots = shots[["match_id", "id", "player", "minute", "possession_team", "shot"]]
shots["xG"] = shots["shot"].apply(lambda x: x["statsbomb_xg"])

In [132]:
# get possession data
possession = events[["match_id", "possession_team", "duration", "type", "location", "minute"]]

### 3. Clean data